<a href="https://colab.research.google.com/github/eceak/Reinforcement-Learning-RL-/blob/main/cart_pole_v0_deep_q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install keras

In [3]:
pip install Adam

In [ ]:
"""
@author: ece
"""

import gym
import numpy as np
'''deep q learning kullanıldığı için keras 
kütüphaneleri aktif edildi'''
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random

class DQLAgent:
     
    def __init__(self, env):
        #parameter/hyperparameter
       
      ###DQN girdisi state_size kadar nöron olduğu için ve 
      ### action_size kadar da çıktı nöron olacağı 
      ###için bunların bilinmesi gerekir.
        self.state_size = env.observation_space.shape[0]
        self.action_size = env.action_space.n
        
        self.gamma = 0.95 #gelecekteki ödülleri belirtir (future reward)
        self.learning_rate = 0.001 #agentın öğrenme hızı
        
        self.epsilon = 1 #explore
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        
        self.memory = deque(maxlen = 1000) #kapasitesi 1000 olan bir liste hafıza için yaratıldı
        
        self.model = self.build_model()
    
    def build_model(self):
        #neural network for deep q learning
        model = Sequential()
        model.add(Dense(48,input_dim = self.state_size,activation = "tanh"))
        model.add(Dense(self.action_size,activation = "linear"))
        model.compile(loss = "mse", optimizer = Adam(lr = self.learning_rate))
        return model
    
    def remember(self,state, action, reward, next_state, done):
        #storage
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        #acting explore or exploit
        if random.uniform(0,1) <= self.epsilon:
            return env.action_space.sample()
        else:
            act_values = self.model.predict(state)
            return np.argmax(act_values[0])
    
    def replay(self,batch_size):
        #training
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            if done:
                target = reward
            else:
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            train_target = self.model.predict(state)
            train_target[0][action] = target
            self.model.fit(state,train_target, verbose = 0)
        
    
    def adaptiveEGreedy(self):
        if self.epsilon > self.epsilon_min :
            self.epsilon *= self.epsilon_decay
        

if __name__ == "__main__":
    
    #initialize gym env and agent
    env = gym.make("CartPole-v0")
    agent = DQLAgent(env)
    
    batch_size = 16
    episodes = 100
    for e in range(episodes):
        
        #initialize environment
        state = env.reset() 
        
        #ilerleyen adımlarda stateleri bir arada tutmak için reshape kullanılır
        state = np.reshape(state, [1,4])
        
        time = 0
        while True:
            
            #act / bir hareket seçiliyor
            action = agent.act(state) #select an action
            
            #step
            next_state, reward, done, _ = env.step(action)
            next_state = np.reshape(next_state, [1,4])
            
            #remember/storage
            agent.remember(state,action, reward, next_state, done)
            
            #update state
            state = next_state
            
            #replay
            agent.replay(batch_size)
            
            #adjust epsilon
            agent.adaptiveEGreedy()
            
            time += 1
            
            if done:
                print("Episode: {}, time: {}".format(e,time))
                break




/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Episode: 0, time: 45
Episode: 1, time: 15
Episode: 2, time: 11
Episode: 3, time: 34
Episode: 4, time: 35
Episode: 5, time: 16
Episode: 6, time: 10
Episode: 7, time: 11
Episode: 8, time: 10
Episode: 9, time: 9
Episode: 10, time: 15
Episode: 11, time: 37
Episode: 12, time: 17
Episode: 13, time: 49
Episode: 14, time: 16
Episode: 15, time: 31
Episode: 16, time: 41
Episode: 17, time: 64
Episode: 18, time: 45
Episode: 19, time: 69
Episode: 20, time: 53
Episode: 21, time: 42
Episode: 22, time: 30
Episode: 23, time: 41
Episode: 24, time: 45
Episode: 25, time: 53
Episode: 26, time: 35
Episode: 27, time: 42
Episode: 28, time: 47
Episode: 29, time: 54
Episode: 30, time: 39
Episode: 31, time: 44
Episode: 32, time: 37
Episode: 33, time: 36
Episode: 34, time: 99
Episode: 35, time: 57
Episode: 36, time: 77
Episode: 37, time: 200
Episode: 38, time: 200
Episode: 39, time: 200
